In [46]:
import torch
from utils.preprocessing import *
from utils.loops import *
from models.vanilla_cnn import *
from models.cnn_attention import *
from models.cnn_gru import *
from models.cnn_lstm import *
from models.cnn_rnn import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
## Set device

# if torch.backends.mps.is_available():
#     device = torch.device("mps")
# else:
#     device = torch.device('cpu')
# print(device)

## When testing, use cpu
device = torch.device('cpu')

In [41]:
## Instantiate Dataloaders
train_dataloader, val_dataloader, test_dataloader = load_data(64)

Shape of training set: (14535, 22, 400)
Shape of validation set: (500, 22, 400)
Shape of training labels: (14535,)
Shape of validation labels: (500,)
Shape of training labels after categorical conversion: (14535, 4)
Shape of validation labels after categorical conversion: (500, 4)
Shape of test labels after categorical conversion: (443, 4)


## Trained on all subjects (0-800)

In [68]:
## Instantiate models and load their weights
models = {}

cnn = CNN(kernel_size=11, pad=5)
checkpoint = torch.load('weights/weights_all_subjects/CNN_epoch60.pt', map_location=torch.device('cpu'))
cnn.load_state_dict(checkpoint['model_state_dict'])
cnn = cnn.to(device)
models['cnn'] = cnn

# rnn = RNN(input_dim=22, conv_dims=[32, 64, 128, 256], hidden_dim=128, num_layers=1)
# checkpoint = torch.load('weights/weights_all_subjects/RNN_epoch46_400_800.pt', map_location=torch.device('cpu'))
# rnn.load_state_dict(checkpoint['model_state_dict'])
# rnn = rnn.to(device)
# models['rnn'] = rnn

# cnn_attention = CNN_Attention_Model()
# checkpoint = torch.load('weights/weights_all_subjects/Attention_epoch100.pt', map_location=torch.device('cpu'))
# cnn_attention.load_state_dict(checkpoint['model_state_dict'])
# cnn_attention = cnn_attention.to(device)
# models['cnn_attention'] = cnn_attention

gru = GRU(input_dim=22, conv_dims=[32, 64, 128], hidden_dim=256, num_layers=1)
checkpoint = torch.load('weights/weights_all_subjects/GRU_epoch18.pt', map_location=torch.device('cpu'))
gru.load_state_dict(checkpoint['model_state_dict'])
gru = gru.to(device)
models['gru'] = gru

lstm = LSTM(input_dim=22, conv_dims=[32, 64, 128], hidden_dim=64, num_layers=1)
checkpoint = torch.load('weights/weights_all_subjects/LSTM_epoch36.pt', map_location=torch.device('cpu'))
lstm.load_state_dict(checkpoint['model_state_dict'])
lstm = lstm.to(device)
models['lstm'] = lstm

In [69]:
## Evaluate ensembled models on test set
# Evaluation criterion: majority vote

# accuracy = test_average(models, test_dataloader, device)
# print('Test Accuracy (average):', accuracy)

accuracy = test_majority(models, test_dataloader, device)
print('Test Accuracy (majority vote):', accuracy)

Test Accuracy (majority vote): 0.6749435665914221


## Trained on all subjects (0-400)